# Pre-procesado de datos

Asignacion de rutas de datos e inicializacion de variables

In [9]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pytesseract
import string

from tqdm import tqdm
from time import sleep

from PIL import Image
from os import listdir, makedirs
from collections import defaultdict
from os.path import join, isdir, splitext

#Kernel
kernel = (3, 3)
#nivel de difuminado
level = 2

#Rutas de datos
images_train = "../dataset/data_train/images/" #raw_path
segment_train = "../dataset/data_train/clasification/" #seg_path
images_test = "../dataset/data_test/images/" 
segment_test = "../dataset/data_test/clasification/" 
images_valid = "../dataset/data_valid/" 
segment_valid = "../dataset/data_valid/" 

#declaracion alfabeto
alphabet = string.digits + string.ascii_letters
#Declaracion del alfabeto reconozible por el programa
recognizer_alphabet = ''.join(sorted(set(alphabet.upper())))

#guardando rutas particulaes de los archivos
files_train = []
files_test = []
files_valid = []
for i in range(3):
    if i == 0:
        for j in listdir(images_train):
            for k in listdir(images_train + j + "/"):
                #0/0_0000.png
                files_train.append(j + "/" + k)
    elif i == 1:
        for j in listdir(images_test):
            for k in listdir(images_test + j + "/"):
                files_test.append(j + "/" + k)
                
    else:
        for j in listdir(images_valid):
            files_valid.append(j + "/" + "x_" + j + ".png")

counts = defaultdict(int)

Instrucciones para tratar cada imagen

In [10]:
for j in range(3):
    index_character = "-1"

    #Se elige que archivos captchas se usaran, su ruta origen y destino
    if j == 0:
        print("Pre-procesando captchas entrenamiento")
        files = files_train
        raw_path = images_train
        seg_path = segment_train
        limit = 1000*4
    elif j == 1:
        print("\nPre-procesando captchas testeo")
        files = files_test
        raw_path = images_test
        seg_path = segment_test
        limit = 150*4
    else:
        print("\nPre-procesando captchas validacion")
        files = files_valid
        raw_path = images_valid
        seg_path = segment_valid
        limit = 1

    for file in files:

        #Se guarda la imagen en escala de grises y se guarda su eqtiqueta
        #Ejemplo: 25_ds45  letters=ds34
        image = cv2.imread(raw_path + file, 0)
        letters = file.split('.')[0].split('_')[1]

        if (index_character != letters[0]) & (j != 2):
            index_character = letters[0]
            print("tratando caracter:", letters[0])

        # blur
        #se crea una matriz de 5x5 con la que se le aplicara el diifuminado a la imagen
        #dst es la imagen difumina
        k = np.ones((5,5),np.float32)/25
        dst = cv2.filter2D(image,-1,k)

        #Se aplica umbral
        #1: Imagen en escala de grises
        #2: valor umbral que se utiliza para clasificar los valores de los píxeles
        #3: Representa el valor que se dará si el valor del píxel es mayor (a veces menor) que el valor de umbral. 
        #4: Es el tipo de umbral que se aplicara
        ret, image = cv2.threshold(dst, 110, 255, cv2.THRESH_BINARY_INV)
        #El método se utiliza para realizar la erosión en la imagen
        #elimina el ruido y pequenos trazos restantes
        image = cv2.erode(image, kernel, iterations = level)

        #Datos de la imagen final
        #devuelve el número de componentes conectados y etiquetado.
        #ademas le da las coordenadas centrales y el rango
        connectivity = 4
        output = cv2.connectedComponentsWithStats(image, connectivity, cv2.CV_32S)


        #asignacion de los datos de la imagen a nuevas variables para su manipulacion
        #Se guardan el umero de etiquetas, etiquetas, estaisticas y centroides de cada imagen en blanco detectada
        num_labels = output[0]
        labels = output[1]
        stats = output[2]
        centroids = output[3]

        #lista de letras 
        objects = []

        for i in range(1, num_labels):
            #verifica el tamanio de la area conectada por mpixeles blancos
            a = stats[i, cv2.CC_STAT_AREA]

            #Si el area es mayor a 50 pixreles lo toma como una letra valida
            if a > 50:
                
                #se guarda los limites de el area en blanco
                x = stats[i, cv2.CC_STAT_LEFT]
                y = stats[i, cv2.CC_STAT_TOP]
                w = stats[i, cv2.CC_STAT_WIDTH]
                h = stats[i, cv2.CC_STAT_HEIGHT]
                

                objects.append((x, y, w, h))

        #organizacion de los datos
        objects.sort(key=lambda t: t[0])

        #numero de datos detectado
        num_detected = min(len(objects), 4)

        #clasificacion de las letras reconocidas
        for i in range(num_detected):
            o = objects[i]
            x = o[0]
            y = o[1]
            w = o[2]
            h = o[3]

            #creacion de la imagen de la letra con filtro en blanco y negro
            #Se corta la imagen en el rango antes obtenido
            img = image[y:y+h, x:x+w]

            letter = letters[i]

            filename = "/" + str(counts[letter]).zfill(6) + ".png"

            #validar su ubicacion de guardado
            if (j != 2):
                if (limit > len(listdir(seg_path + letter))):
                    path = seg_path + letter  + filename
                    cv2.imwrite(path, img)
            else:
                path_to_clasification=seg_path + "".join(letters) + "/clasification/"+str(i)+'_'+letter
                if (limit > len(listdir(path_to_clasification))):
                    path = path_to_clasification + filename  
                    cv2.imwrite(path, img)

            counts[letter] += 1
print("\n\nlisto")

Pre-procesando captchas entrenamiento
tratando caracter: 0
tratando caracter: 1
tratando caracter: 2
tratando caracter: 3
tratando caracter: 4
tratando caracter: 5
tratando caracter: 6
tratando caracter: 7
tratando caracter: 8
tratando caracter: 9
tratando caracter: A
tratando caracter: B
tratando caracter: C
tratando caracter: D
tratando caracter: E
tratando caracter: F
tratando caracter: G
tratando caracter: H
tratando caracter: I
tratando caracter: J
tratando caracter: K
tratando caracter: L
tratando caracter: M
tratando caracter: N
tratando caracter: O
tratando caracter: P
tratando caracter: Q
tratando caracter: R
tratando caracter: S
tratando caracter: T
tratando caracter: U
tratando caracter: V
tratando caracter: W
tratando caracter: X
tratando caracter: Y
tratando caracter: Z

Pre-procesando captchas testeo
tratando caracter: 0
tratando caracter: 1
tratando caracter: 2
tratando caracter: 3
tratando caracter: 4
tratando caracter: 5
tratando caracter: 6
tratando caracter: 7
tratan